TODO: na grafoch ukazat pocet jednotlivich unikatnych hodnot pre kazdu categorical hodnotu

In [1]:
import pandas as pd
import numpy as np
import json
from joblib import dump

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# depth of this filr in the project
file_depth = '../..'

In [3]:
with open(file_depth + '/config/data_1m_config.json') as config_file:
    config = json.load(config_file)

context_data_path = config['context_data']

In [4]:
context_data = pd.read_csv(file_depth + context_data_path)
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,user_zip_code,movie_year
0,1,1193,7,0,4,4,new_years,F,1,10,48067,1975
1,1,661,7,0,4,4,new_years,F,1,10,48067,1996
2,1,914,7,0,4,4,new_years,F,1,10,48067,1964
3,1,3408,7,0,4,4,new_years,F,1,10,48067,2000
4,1,2355,7,0,4,4,no_holiday,F,1,10,48067,1998
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,no_holiday,M,25,6,11106,1989
1000205,6040,1094,3,1,1,4,no_holiday,M,25,6,11106,1992
1000206,6040,562,3,1,1,4,no_holiday,M,25,6,11106,1995
1000207,6040,1096,3,1,1,4,no_holiday,M,25,6,11106,1982


In [6]:
# Label encode other categorical columns
categorical_columns = ['holiday', 'user_gender']
for column in categorical_columns:
    context_data[column] = LabelEncoder().fit_transform(context_data[column])
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,user_zip_code,movie_year
0,1,1193,7,0,4,4,1,0,1,10,48067,1975
1,1,661,7,0,4,4,1,0,1,10,48067,1996
2,1,914,7,0,4,4,1,0,1,10,48067,1964
3,1,3408,7,0,4,4,1,0,1,10,48067,2000
4,1,2355,7,0,4,4,2,0,1,10,48067,1998
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,2,1,25,6,11106,1989
1000205,6040,1094,3,1,1,4,2,1,25,6,11106,1992
1000206,6040,562,3,1,1,4,2,1,25,6,11106,1995
1000207,6040,1096,3,1,1,4,2,1,25,6,11106,1982


### Split data and save then

Load target (ratings)

In [7]:
target_data_path = config['target_data']
target_ratings = pd.read_csv(file_depth + target_data_path)
target_ratings

,rating_id,rating
0,0,5
1,1,3
2,2,3
3,3,4
4,4,5
...,...,...
1000204,1000204,1
1000205,1000205,5
1000206,1000206,5
1000207,1000207,4


Merge ratings with context_data

In [8]:
context_data['rating_id'] = context_data.index
context_data = context_data.merge(target_ratings, on='rating_id', how='left')
context_data = context_data.drop(['rating_id'], axis=1)
context_data

,user_id,movie_id,day,is_weekday,season,part_of_day,holiday,user_gender,user_age,user_occupation,user_zip_code,movie_year,rating
0,1,1193,7,0,4,4,1,0,1,10,48067,1975,5
1,1,661,7,0,4,4,1,0,1,10,48067,1996,3
2,1,914,7,0,4,4,1,0,1,10,48067,1964,3
3,1,3408,7,0,4,4,1,0,1,10,48067,2000,4
4,1,2355,7,0,4,4,2,0,1,10,48067,1998,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,3,1,1,4,2,1,25,6,11106,1989,1
1000205,6040,1094,3,1,1,4,2,1,25,6,11106,1992,5
1000206,6040,562,3,1,1,4,2,1,25,6,11106,1995,5
1000207,6040,1096,3,1,1,4,2,1,25,6,11106,1982,4


In [9]:
train, temp = train_test_split(context_data, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

In [10]:
train_target_ratings = train['rating'].values
val_target_ratings = val['rating'].values
test_target_ratings = test['rating'].values

train_data = train.drop(['rating'], axis=1)
val_data = val.drop(['rating'], axis=1)
test_data = test.drop(['rating'], axis=1)

In [11]:
# Standardize columns
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.fit_transform(val_data)
test_data = scaler.fit_transform(test_data)

In [12]:
training_data_path = config['training_data']

# Save the fitted StandardScaler to disk
dump(scaler, file_depth + training_data_path + 'data/scaler.pkl')

# Save the data to disk
np.save(file_depth + training_data_path + 'data/train_data.npy', train_data)
np.save(file_depth + training_data_path + 'data/val_data.npy', val_data)
np.save(file_depth + training_data_path + 'data/test_data.npy', test_data)
np.save(file_depth + training_data_path + 'data/train_target_ratings.npy', train_target_ratings)
np.save(file_depth + training_data_path + 'data/val_target_ratings.npy', val_target_ratings)
np.save(file_depth + training_data_path + 'data/test_target_ratings.npy', test_target_ratings)

## NN MODEL

In [22]:
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from keras.optimizers import Adam
from keras.losses import MeanSquaredError as LossMSE
from keras.metrics import MeanSquaredError, MeanAbsoluteError

import tensorflow as tf
import plotly.express as px

In [23]:
model = Sequential()
model.add(Dense(64, input_shape=(train_data.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(1))
# model.add(Activation('linear'))

In [25]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 activation_2 (Activation)   (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 8)                 2

#### Training

In [26]:
with open(file_depth + '/nn_config.json') as config_file:
    nn_config = json.load(config_file)

In [27]:
batch_size = int(nn_config['batch_size'])
epochs = int(nn_config['epochs'])
patience = int(nn_config['patience'])
learning_rate = float(nn_config['learning_rate'])
early_stopping = EarlyStopping(patience=patience, monitor='val_loss')

In [28]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     learning_rate,
#     decay_steps=75,
#     decay_rate=0.96,
#     staircase=True)

In [29]:
# scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
# callbacks = [early_stopping, scheduler, WandbMetricsLogger()]
# callbacks = [early_stopping, WandbMetricsLogger()]
# callbacks = [early_stopping, scheduler]
callbacks = [early_stopping]

In [30]:
# model.compile(optimizer=Adam(learning_rate=lr), loss=LossMSE(), metrics=[MeanSquaredError(), MeanAbsoluteError()])
model.compile(optimizer=Adam(), loss=LossMSE(), metrics=[MeanSquaredError(), MeanAbsoluteError()])

In [31]:
# trained_model = model.fit(train_data, train_target_ratings, batch_size, epochs, validation_data=(val_data, val_target_ratings), callbacks=callbacks, verbose=1)
trained_model = model.fit(train_data, train_target_ratings, batch_size, epochs, validation_data=(val_data, val_target_ratings), callbacks=callbacks, verbose=1)

Epoch 1/500
12503/12503 [==============================] - 39s 3ms/step - loss: 1.2158 - mean_squared_error: 1.2158 - mean_absolute_error: 0.9025 - val_loss: 1.1802 - val_mean_squared_error: 1.1802 - val_mean_absolute_error: 0.8921
Epoch 2/500
12503/12503 [==============================] - 35s 3ms/step - loss: 1.1672 - mean_squared_error: 1.1672 - mean_absolute_error: 0.8840 - val_loss: 1.1653 - val_mean_squared_error: 1.1653 - val_mean_absolute_error: 0.8813
Epoch 3/500
12503/12503 [==============================] - 30s 2ms/step - loss: 1.1497 - mean_squared_error: 1.1497 - mean_absolute_error: 0.8751 - val_loss: 1.1526 - val_mean_squared_error: 1.1526 - val_mean_absolute_error: 0.8771
Epoch 4/500
12503/12503 [==============================] - 29s 2ms/step - loss: 1.1355 - mean_squared_error: 1.1355 - mean_absolute_error: 0.8677 - val_loss: 1.1380 - val_mean_squared_error: 1.1380 - val_mean_absolute_error: 0.8683
Epoch 5/500
12503/12503 [==============================] - 31s 2ms/step 

In [32]:
# my_model.save('./Outputs/best_model/')
# model.save('../training/outputs/')

model.save(file_depth + training_data_path + 'outputs/best_model/model.h5')

d:\.exe\miniconda\envs\dp\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
fig_loss = px.line({
    'Train': trained_model.history['loss'],
    'Val': trained_model.history['val_loss']
    },
    title='Train vs Val Loss')
fig_loss.show()

In [34]:
fig_mae = px.line({
    'Train': trained_model.history['mean_squared_error'],
    'Val': trained_model.history['val_mean_squared_error']
    },
    title='Train vs Val MSE')
fig_mae.show()

In [35]:
fig_mae = px.line({
    'Train': trained_model.history['mean_absolute_error'],
    'Val': trained_model.history['val_mean_absolute_error']
    },
    title='Train vs Val MAE')
fig_mae.show()

In [36]:
# Validation data values in the last epoch
print("number of epochs:", trained_model.epoch[-1] + 1)
print("val_loss:", trained_model.history['val_loss'][-1])
print("val_mean_squared_error:", trained_model.history['val_mean_squared_error'][-1])
print("val_mean_absolute_error:", trained_model.history['val_mean_absolute_error'][-1])

number of epochs: 500
val_loss: 1.0467556715011597
val_mean_squared_error: 1.0467556715011597
val_mean_absolute_error: 0.8187956213951111


In [37]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
test_loss, test_mean_squared_error, test_mean_absolute_error = model.evaluate(test_data, test_target_ratings)
print("test_loss:", test_loss)
print("test_mean_squared_error:", test_mean_squared_error)
print("test_mean_absolute_error:", test_mean_absolute_error)
print('\n')

print("generate predictions for 10 samples")
predictions = model.predict(test_data)
test_y_comparison = pd.DataFrame({'test_y_true': test_target_ratings, 'test_y_pred': [x for x in predictions]})

print(test_y_comparison.head(10))

Evaluate on test data
3126/3126 [==============================] - 6s 2ms/step - loss: 1.0464 - mean_squared_error: 1.0464 - mean_absolute_error: 0.8197
test_loss: 1.0463882684707642
test_mean_squared_error: 1.0463882684707642
test_mean_absolute_error: 0.819678008556366


generate predictions for 10 samples
3126/3126 [==============================] - 6s 2ms/step
   test_y_true  test_y_pred
0            4  [2.8987107]
1            3   [3.073485]
2            5  [3.9234102]
3            5  [2.9328077]
4            4  [3.7625983]
5            4   [3.855852]
6            4  [4.2630057]
7            3   [4.073013]
8            1  [3.1256104]
9            5   [4.193324]
